<a href="https://colab.research.google.com/github/PadmarajBhat/Rapids.AI/blob/master/rapids_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [2]:
!nvidia-smi

Sun Jul 21 14:47:30 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
print(device_name)
if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

b'Tesla T4'
Woo! You got the right kind of GPU!


#Setup:

1. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
2. Install RAPIDS libraries
3. Set necessary environment variables
4. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions

In [4]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local -c nvidia -c rapidsai \
  -c numba -c conda-forge -c defaults nvstrings=0.8 python=3.6 cudatoolkit=10.0

!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install  -q -y --prefix /usr/local dask

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

--2019-07-21 14:47:39--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M  95.8MB/s    in 0.6s    

2019-07-21 14:47:40 (95.8 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls -l "/content/drive/My Drive/Colab Notebooks/TrainData_PA.csv"

-rw------- 1 root root 5493629 Jul 16 23:19 '/content/drive/My Drive/Colab Notebooks/TrainData_PA.csv'


In [7]:
import dask.dataframe as dd
import numpy as np
tips_df = dd.read_csv("/content/drive/My Drive/Colab Notebooks/TrainData_PA.csv", assume_missing=True)
tips_df.head()

,county,city,zipcode,address,state,rent,latitude,longitude,cemetery_dist_miles,nationalhighway_miles,railline_miles,starbucks_miles,walmart_miles,hospital_miles,physician_dist_miles,dentist_dist_miles,opt_dist_miles,vet_dist_miles,farmers_miles,time,bed,bath,halfbath,sqft,property_type,garage,yearbuilt,pool,fireplace,patio,lotsize,Census_MedianIncome,CollegeGrads,WhiteCollar,Schools,Unemployment,EmploymentDiversity,Census_Vacancy,Crime_Rate,HomePrice
0,NaN,WEXFORD,NaN,266 Clematis Dr Allegheny County,PA,2400.0,40.618200,-80.077600,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2016.25,3.0,2.0,1.0,2000.0,Condo,1.0,2008.0,0.0,1.0,0.0,4086.388045,54476.09,21.0,66.57,48.3,5.1,3.48,3.42,2.4,158051.0
1,NaN,WHITEHALL,NaN,2310 N 1st Ave Lehigh County,PA,995.0,40.649906,-75.478940,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2016.25,2.0,1.0,1.0,1100.0,Condo,0.0,1935.0,0.0,0.0,0.0,2247.513425,54476.09,21.0,66.57,48.3,5.1,3.48,3.42,2.4,158051.0
2,NaN,WHITEHALL,NaN,3338 St Stephens Ln Lehigh County,PA,1740.0,40.646282,-75.510056,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2015.75,3.0,2.0,1.0,1522.0,Condo,0.0,2006.0,0.0,1.0,1.0,3109.741302,54476.09,21.0,66.57,48.3,5.1,3.48,3.42,2.4,158051.0
3,NaN,WAYNESBORO,NaN,97 W Main St Franklin County,PA,675.0,39.756992,-77.579704,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2016.25,3.0,1.0,1.0,1150.0,Condo,0.0,1960.0,0.0,0.0,0.0,2349.673126,54476.09,21.0,66.57,48.3,5.1,3.48,3.42,2.4,158051.0
4,NaN,QUAKERTOWN,NaN,200 E Broad St Bucks County,PA,1300.0,40.441177,-75.332540,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2016.25,3.0,2.0,1.0,1000.0,SFR,0.0,1960.0,0.0,0.0,0.0,2043.194023,54476.09,21.0,66.57,48.3,5.1,3.48,3.42,2.4,158051.0


https://docs.dask.org/en/latest/install.html - for installation and other doc

In [14]:
for i in tips_df.HomePrice.compute():
  print(i)

158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
158051.0
166560.0
158051.0
102911.0
158051.0
215037.0
158051.0
143206.0
158051.0
158051.0
158051.0
158051.0
202774.0
158051.0
158051.0
158051.0
158051.0
288879.0
158051.0
158051.0
158051.0
132290.0
137828.0
158051.0
158051.0
158051.0
158051.0
467479.0
158051.0
265566.0
158051.0
128924.0
158051.0
128924.0
158051.0
158051.0
160956.0
158051.0
188855.0
158051.0
153246.0
158051.0
186974.0
158051.0
158051.0
159899.0
142846.0
179375.0
158051.0
158051.0
124724.0
158051.0
69321.0
158051.0
81615.0
158051.0
158051.0
132738.0
57748.0
158051.0
253070.0
607915.0
158051.0
460421.0
100670.0
158051.0
72345.0
47140.0
178178.0
158051.0
85555.0
158051.0
69545.0
101204.0
72575.0
120965.0
158051.0
504870.0
101831.0
158051.0
238712.0
158051.0
71656.0
309268.0
158051.0
4

In [6]:
tips_df.select_dtypes(include=np.number).dtypes

zipcode                  float64
rent                     float64
latitude                 float64
longitude                float64
cemetery_dist_miles      float64
nationalhighway_miles    float64
railline_miles           float64
starbucks_miles          float64
walmart_miles            float64
hospital_miles           float64
physician_dist_miles     float64
dentist_dist_miles       float64
opt_dist_miles           float64
vet_dist_miles           float64
farmers_miles            float64
time                     float64
bed                      float64
bath                     float64
halfbath                 float64
sqft                     float64
garage                   float64
yearbuilt                float64
pool                     float64
fireplace                float64
patio                    float64
lotsize                  float64
Census_MedianIncome      float64
CollegeGrads             float64
WhiteCollar              float64
Schools                  float64
Unemployme

In [33]:
print(list(tips_df['HomePrice']))

NotImplementedError: ignored

In [25]:
tips_df['zipcode']=tips_df.zipcode.astype(np.int64)
#tips_df['zipcode']=tips_df.zipcode.astype(np.float64)
tips_df.select_dtypes(include=np.number).dtypes
tips_df.head()

ValueError: ignored

In [26]:
#tips_df['zipcode']=tips_df.zipcode.astype(np.int64)
tips_df['zipcode']=tips_df.zipcode.astype(np.float64)
tips_df.select_dtypes(include=np.number).dtypes
tips_df.head()

ValueError: ignored

In [28]:
tips_df.npartitions

1

In [29]:
tips_df.divisions

(None, None)

### Example of delayed function:

Here the map to integer did not happen at the read_csv line 

In [34]:
print(tips_df.shape)
print(list(tips_df[tips_df.yearbuilt.isna()]["yearbuilt"]))
tips_df['yearbuilt']=tips_df.yearbuilt.fillna(0.0)
tips_df.head()
tips_df = tips_df.set_index("yearbuilt")

(Delayed('int-ee25b1e8-8473-43ed-a32f-1309a7404e72'), 40)


ValueError: ignored

In [36]:
list(tips_df.yearbuilt)

ValueError: ignored

In [37]:
tips_df.yearbuilt.head()

ValueError: ignored

In [39]:
tips_df.HomePrice.head()

ValueError: ignored

In [43]:
print(tips_df.info())

<class 'dask.dataframe.core.DataFrame'>
Columns: 40 entries, county to HomePrice
dtypes: object(5), float64(34), int64(1)None


#[cuML](https://github.com/rapidsai/cuml)#

This snippet loads a 

As above, all calculations are performed on the GPU.

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-extended